In [2]:
import sys, os, json, copy, sklearn, shap, gc, joblib, copy, time
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
import pickle as pkl
from tqdm import tqdm
from smart_open import open
import seaborn as sns
from sklearn.metrics import roc_curve,roc_auc_score, precision_score, recall_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from rdsutils.woe import WOE_Transform                                             
from rdsutils.metrics import get_pred_reports

pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', None)


%load_ext autoreload
%autoreload 2

print(lgb.__version__)

3.2.1


### for development period data

In [3]:
config_path = "s3://sofi-data-science/hpeng/pl-gen4/artifact/config.json"
artifact_path = "s3://sofi-data-science/hpeng/pl-gen4/artifact"
with open(os.path.join(config_path), "r") as f:
    config = json.load(f)

In [4]:
exp_dict = pd.read_csv(config["meta"]["exp_dict_path"])
af_dict = pd.read_csv(config["meta"]["af_member_cust_dict_path"])

In [11]:
feature_list = ['id','coborrower_indicator','applicant_type','date_start','date_fund','is_funded','income_update','all_time_months_oldest_pl_trade',
                                   'all_time_lending_member_flag','current_pl_trade_bal_ratio','current_pl_trade_current_prin',
                                  'current_pl_trade_orig_bal'
]

In [12]:
df_dev1_full = pd.read_parquet(config["data"]["clean"]["all_features_dev1"], columns = feature_list)
df_dev2_full = pd.read_parquet(config["data"]["clean"]["all_features_dev2"], columns = feature_list)
df_dev_full = pd.concat([df_dev1_full, df_dev2_full], axis=0, ignore_index=True)

In [22]:
df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='N') & (df_dev_full.is_funded==1) & (~df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(10)

,id,coborrower_indicator,applicant_type,date_start,date_fund,is_funded,income_update,all_time_months_oldest_pl_trade,all_time_lending_member_flag,current_pl_trade_bal_ratio,current_pl_trade_current_prin,current_pl_trade_orig_bal
140,4792136,N,PRIMARY,2017-10-05,2017-10-10,1,104549.12,4.0,1,0.972870,38914.82,40000.00
750,4953639,N,PRIMARY,2017-10-22,2017-11-03,1,110744.00,12.0,1,0.882603,35304.14,40000.00
827,4809196,N,PRIMARY,2017-10-07,2018-01-04,1,158399.42,13.0,1,0.854883,41015.58,47978.00
847,4931933,N,PRIMARY,2017-10-19,2017-10-25,1,144945.00,20.0,1,NaN,NaN,NaN
1289,4871377,N,PRIMARY,2017-10-13,2017-10-19,1,120359.98,27.0,1,NaN,NaN,NaN
1416,4757355,N,PRIMARY,2017-10-01,2017-10-11,1,84058.46,10.0,1,0.907852,19233.00,21185.16
1504,4834764,N,PRIMARY,2017-10-10,2017-10-12,1,104731.57,4.0,1,0.461720,16099.72,34869.00
1705,4888571,N,PRIMARY,2017-10-15,2017-10-18,1,90000.00,21.0,1,0.504960,5487.60,10867.39
1707,4946250,N,PRIMARY,2017-10-21,2017-11-01,1,55139.37,9.0,1,0.858202,12169.30,14180.00
1869,5255322,N,PRIMARY,2017-12-03,2017-12-08,1,134792.76,22.0,1,0.701212,35060.59,50000.00


In [32]:
df1 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='N') & (df_dev_full.is_funded==1) & (~df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df2 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='N') & (df_dev_full.is_funded==1) & (df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df3 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='N') & (df_dev_full.is_funded==0) & (~df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df4 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='N') & (df_dev_full.is_funded==0) & (df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df5 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='Y') & (df_dev_full.is_funded==1) & (~df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df6 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='Y') & (df_dev_full.is_funded==1) & (df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df7 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='Y') & (df_dev_full.is_funded==0) & (~df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df8 = df_dev_full[df_dev_full.id.isin(df_dev_full[(df_dev_full.coborrower_indicator=='Y') & (df_dev_full.is_funded==0) & (df_dev_full.all_time_months_oldest_pl_trade.isnull())].id.to_list())].head(125)
df_dev_tie_out = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0, ignore_index=True)

In [36]:
df_dev_tie_out.to_parquet('s3://sofi-data-science/hpeng/pl-gen4/artifact/df_dev_inputs_tie_out.parquet')

In [34]:
df_dev_tie_out.to_csv('test_dev.csv')

### for 2022.01 data

In [29]:
df_recent = pd.read_parquet('s3://sofi-data-science/hpeng/pl-gen4/artifact/df_202201_inputs_tie_out.parquet')

In [30]:
df_recent[(df_recent.date_start>='2022-01-01') & (df_recent.date_start<='2022-01-15')][['id','coborrower_indicator','applicant_type','date_start','date_fund','income_update','all_time_months_oldest_pl_trade',
                                   'all_time_lending_member_flag','current_pl_trade_bal_ratio','current_pl_trade_current_prin',
                                  'current_pl_trade_orig_bal']].to_csv('gen4_implementation_inputs.csv')